In [1]:
#Google Colab File -ashutoshtiwari13

import sys
import gym 
import numpy as np
from collections import defaultdict 

env= gym.make('Blackjack-v0')

In [3]:
print(env.observation_space)
print(env.action_space)

Tuple(Discrete(32), Discrete(11), Discrete(2))
Discrete(2)


In [7]:
### Random Policy 
for episodes in range(3):
  state = env.reset()
  while True:
    print(state)
    action = env.action_space.sample() 
    state, reward , done, info = env.step(action)
    if done:
      print("End Game : Reward -", reward)
      print("You won ! Hola :D") if reward > 0 else print("You lost ! :(")
      break

(15, 6, False)
End Game : Reward - 1.0
You won ! Hola :D
(20, 3, False)
End Game : Reward - -1.0
You lost ! :(
(8, 7, False)
(18, 7, False)
(20, 7, False)
End Game : Reward - -1.0
You lost ! :(


In [18]:
## Monte-Carlo Prediction (For the present env , First visit and every visit result in the same solution)
##Stochastic episode generation  

def generate_episode(env_instance):
  for episodes in range(3):
    episode =[]
  state= env.reset()
  while True:
    probs  = [0.8, 0.2] if state[0] > 18 else [0.2,0.8]
    action = np.random.choice(np.arange(2),p =probs)
    next_state , reward,done , info = env.step(action)
    episode.append((state, action , reward))
    state = next_state
    if done:
      break
  
  return episode   

In [24]:
##Monte-carlo every visit Control 

def every_visit_monte_carlo_control(env , num_episodes , generate_episode, gamma =1.0):
  total_sum = defaultdict(lambda: np.zeros(env.action_space.n))
  iters = defaultdict(lambda: np.zeros(env.action_space.n))
  Q = defaultdict(lambda: np.zeros(env.action_space.n))


  for episodes in range(1, num_episodes+1):
    if episodes %10000 ==0:
      print("\rEpisode {}/{}.".format(episodes,num_episodes),end = "")
    episode = generate_episode(env)
    states , actions, rewards = zip(*episode)
  
    #consideing the case of discounting 
    discounts = np.array([gamma ** i for i in range(len(rewards)+1)])

    for i , state in enumerate(states):

      total_sum[state][actions[i]] += sum(rewards[i:]* discounts[:-(1+i)])
      iters[states][actions[i]] += 1.0
      Q[states][actions[i]] += total_sum[state][actions[i]]/iters[states][actions[i]]
  return Q     

In [25]:
#result
Q = every_visit_monte_carlo_control(env, 500000,generate_episode)
print(Q)

Episode 500000/500000.defaultdict(<function every_visit_monte_carlo_control.<locals>.<lambda> at 0x7f37b2e48488>, {((15, 4, False),): array([ -211.5384533 , -2141.84337254]), ((20, 4, False),): array([ 3319.12240923, -1061.00808235]), ((14, 1, False), (21, 1, False)): array([ 1254.33752036, -1939.14620806]), ((14, 4, False),): array([ -146.58424938, -2054.81370324]), ((12, 10, False),): array([-2057.04838232, -7407.03103869]), ((21, 10, True),): array([6024.41344826,    0.        ]), ((5, 10, False), (8, 10, False), (15, 10, False)): array([-1.00000000e+00, -3.12946812e+03]), ((10, 2, False), (13, 2, False)): array([-191.59184149, -632.36099324]), ((9, 10, False), (18, 10, False)): array([ -587.99820637, -5700.93903446]), ((10, 1, False), (18, 1, False)): array([ -314.43911791, -1549.07181367]), ((20, 9, True),): array([452.99880036,   0.        ]), ((13, 1, False),): array([ -740.6457963 , -2379.40195501]), ((19, 6, False),): array([1757.61129762, -708.85855005]), ((12, 7, False), (21